In [2]:
import pandas as pd
import numpy as np

# Load the train and test datasets
train_file_path = '/root/code/final_train.csv'
test_file_path = '/root/code/final_test.csv'
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# Remove the unnamed column if exists
train_data = train_data.loc[:, ~train_data.columns.str.contains('^Unnamed')]
test_data = test_data.loc[:, ~test_data.columns.str.contains('^Unnamed')]

# Convert categorical variables to numerical where appropriate for correlation analysis
train_data_numeric = train_data.select_dtypes(include=[np.number])
test_data_numeric = test_data.select_dtypes(include=[np.number])

# Compute the correlation matrix
correlation_matrix = train_data_numeric.corr()

# Identify highly correlated features (absolute value > 0.8) to drop
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column].abs() > 0.8)]

# Find the feature most correlated with 'target'
target_correlation = correlation_matrix['target'].drop('target')
most_correlated_with_target = target_correlation.abs().idxmax()
most_correlated_value = target_correlation[most_correlated_with_target]

# Identify columns highly correlated with 'target'
threshold = 0.8
highly_correlated_with_target = target_correlation[abs(target_correlation) > threshold].index.tolist()

# Filter out columns that are highly correlated with 'target'
remaining_columns = train_data_numeric.columns.difference(highly_correlated_with_target + ['target'])

# Compute the correlation matrix for the remaining columns
remaining_correlation_matrix = train_data_numeric[remaining_columns].corr()

# Identify columns to drop based on high correlation within the remaining variables
correlation_threshold = 0.8
upper_triangle_remaining = remaining_correlation_matrix.where(np.triu(np.ones(remaining_correlation_matrix.shape), k=1).astype(bool))
to_drop_remaining = [column for column in upper_triangle_remaining.columns if any(upper_triangle_remaining[column].abs() > correlation_threshold)]

# Drop the identified columns from the train dataset
cleaned_train_data = train_data_numeric.drop(columns=to_drop_remaining + highly_correlated_with_target)

# Apply the same columns removal to the test dataset
cleaned_test_data = test_data_numeric.drop(columns=to_drop_remaining + highly_correlated_with_target, errors='ignore')

# Save the cleaned data to new CSV files if needed
cleaned_train_data.to_csv('cleaned_train_data.csv', index=False)
cleaned_test_data.to_csv('cleaned_test_data.csv', index=False)

print("Columns to drop:", to_drop_remaining + highly_correlated_with_target)
print("Most correlated with target:", most_correlated_with_target, most_correlated_value)


/tmp/ipykernel_525683/3644729738.py:7: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(train_file_path)
/tmp/ipykernel_525683/3644729738.py:8: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(test_file_path)


Columns to drop: ['Job_Platform', 'Life_Redesign_Education_Consulting_Social_Participation', 'Minor_Population', 'Minor_Population_Growth_Rate', 'Outflow_Population', 'Outflow_Population_Growth_Rate', 'Senior_Population', 'Senior_Population_Growth_Rate', 'Social_Contribution_Academy', 'call_rate_1day_avg', 'cancellation_reason_date', 'cd_91_day', 'contract_year_month', 'corporate_bond_3yr_aa_minus', 'floor_log', 'general_food', 'government_bond_10yr', 'government_bond_3yr', 'government_bond_5yr', 'maintenance_area', 'residential_area', 'rest_area_food', 'rounded_X', 'rounded_Y', 'total_households']
Most correlated with target: exclusive_area 0.5770410539850063


In [7]:
# Check for missing values
missing_values = cleaned_train_data.isnull().sum()

# Filter columns with more than 700,000 missing values
columns_with_many_missing = missing_values[missing_values > 200000]

# Display the result
print("Columns with more than 700,000 missing values:")
print(columns_with_many_missing)

Columns with more than 700,000 missing values:
complex_description              1050240
total_buildings                   870630
gross_area                        869563
household_status_below_60sqm      869608
household_status_60_to_85sqm      869608
household_status_85_to_135sqm     869608
household_status_above_135sqm    1118495
building_area                     869714
parking_spaces                    869714
cinema_distance                   869670
large_store_distance              869670
Female_Population                 895632
Female_Population_Growth_Rate     895632
dtype: int64


In [8]:
# Drop these columns
cleaned_train_data_dropped = cleaned_train_data.drop(columns=columns_with_many_missing.index)

# Display the result
print("DataFrame after dropping columns with more than 700,000 missing values:")
print(cleaned_train_data_dropped)

DataFrame after dropping columns with more than 700,000 missing values:
         main_number  sub_number  exclusive_area  contract_day  floor  \
0              658.0         1.0           79.97             8      3   
1              658.0         1.0           79.97            22      4   
2              658.0         1.0           54.98            28      5   
3              658.0         1.0           79.97             3      4   
4              658.0         1.0           79.97             8      2   
...              ...         ...             ...           ...    ...   
1118817        382.0         0.0           59.94            12     11   
1118818        382.0         0.0           59.94            25     10   
1118819        382.0         0.0           84.83            31     20   
1118820        382.0         0.0           84.83            15      8   
1118821         11.0        67.0           52.46            10      5   

         construction_year  target  base_rate  with

In [9]:
cleaned_train_data_dropped.columns


Index(['main_number', 'sub_number', 'exclusive_area', 'contract_day', 'floor',
       'construction_year', 'target', 'base_rate', 'within_500m_bus',
       'nearest_distance', 'within_250m', 'within_500m', 'beyond_500m',
       'distance_elementary', 'distance_middle', 'distance_elem50',
       'distance_elem75', 'distance_elem100', 'distance_middle50',
       'distance_middle75', 'distance_middle100', 'daechi_mokdong',
       'contract_month', 'high_achivement', 'coordinate_x', 'coordinate_y',
       'Libraries_By_Coordinate', 'Libraries_By_District', 'Education_For_50+',
       'Policy_Planning_For_50+', 'Operation', 'Support', 'Community_Support',
       'Total_Bike_Racks_By_District', 'Total_Bike_Racks_By_Coordinate',
       'Inflow_Population', 'Inflow_Population_Growth_Rate'],
      dtype='object')

In [10]:
columns_to_drop = ['main_number', 'sub_number']
final = cleaned_train_data_dropped.drop(columns=columns_to_drop)

In [16]:
final.shape

(1118822, 35)

In [12]:
final.to_csv('/root/base_data/final_train_dropcolumns.csv')

In [15]:
columns_to_select = [
    'exclusive_area', 'contract_day', 'floor', 'construction_year', 
    'base_rate', 'within_500m_bus', 'nearest_distance', 'within_250m', 'within_500m', 
    'beyond_500m', 'distance_elementary', 'distance_middle', 'distance_elem50', 
    'distance_elem75', 'distance_elem100', 'distance_middle50', 'distance_middle75', 
    'distance_middle100', 'daechi_mokdong', 'contract_month', 'high_achivement', 
    'coordinate_x', 'coordinate_y', 'Libraries_By_Coordinate', 'Libraries_By_District', 
    'Education_For_50+', 'Policy_Planning_For_50+', 'Operation', 'Support', 
    'Community_Support', 'Total_Bike_Racks_By_District', 'Total_Bike_Racks_By_Coordinate', 
    'Inflow_Population', 'Inflow_Population_Growth_Rate'
]

# Select the specified columns
selected_test_data = test_data[columns_to_select]

# Display the result
print("Selected columns from test_data:")
print(selected_test_data)


Selected columns from test_data:
      exclusive_area  contract_day  floor  construction_year  base_rate  \
0            79.9700          26.0    5.0             1987.0        3.5   
1           108.2017          15.0   10.0             2021.0        3.5   
2           161.0000          28.0   15.0             1984.0        3.5   
3           133.4600          10.0   14.0             1984.0        3.5   
4           104.4300          18.0    6.0             1984.0        3.5   
...              ...           ...    ...                ...        ...   
9267         84.6500          19.0   13.0             2014.0        3.5   
9268         84.6200          25.0   12.0             2014.0        3.5   
9269        101.6500          27.0   12.0             2014.0        3.5   
9270         84.9400           2.0   18.0             2014.0        3.5   
9271         84.6500           4.0   13.0             2014.0        3.5   

      within_500m_bus  nearest_distance  within_250m  within_500m 

In [17]:
selected_test_data.to_csv('/root/base_data/final_test_dropcolumns.csv')